# Counting Stability Chambers of Moduli Spaces

This notebook shows how this partitioning algorithm is useful in **multiple areas of interest**.
In particular, it has been used in **state-of-the-art research on moduli spaces of parabolic vector bundles**,
where we used it to **count the number of stability chambers** for moduli spaces with parameters $n$ (number of stability points) and $r$ (rank of vector bundles).

It has been applied across a **wide range of $(n, r)$**, leading to **different shapes and dimensionalities** (up to **7**), and **numbers of hyperplanes** (up to **~130**).

For background see:  
> [*A computational analysis of isomorphism classes of moduli spaces of parabolic vector bundles*](https://repositorio.comillas.edu/xmlui/handle/11531/89959)


## Step 1. Imports and Setup

In [7]:
import numpy as np
import itertools
import math
from fractions import Fraction
from time import perf_counter

from polypart.geometry import Polytope, Hyperplane
from polypart.ppart import build_partition_tree
from polypart.ftyping import as_fraction_vector
from polypart.io import save_tree



## Step 2. Helper Functions

We define functions to:
- Construct the inequalities describing our initial polytope (product of simplices),
- Generate the candidate hyperplanes (stability walls) that divide the space into chambers.


In [2]:
def generate_admissible_matrices_fixed_r_prime(n: int, r: int, r_prime: int, remove_even_symetry: bool = False):
    combs = itertools.combinations(range(r), r_prime)
    variations = itertools.product(combs, repeat=n)
    N = math.comb(r, r_prime) ** n
    for i, variation in enumerate(variations):
        if remove_even_symetry and r == 2 * r_prime and i >= N // 2:
            return
        n_ = np.zeros((n, r), dtype=int)
        for ii in range(n):
            for jj in variation[ii]:
                n_[ii, jj] = 1
        yield n_

def get_plane_intercept_bounds(w: np.ndarray):
    w = w[:, ::-1]
    cumsums = np.cumsum(w, axis=1)
    cumsums = np.hstack((np.zeros_like(cumsums[:, :1]), cumsums))
    lower_bound = cumsums.min(axis=1).sum()
    upper_bound = cumsums.max(axis=1).sum()
    return lower_bound, upper_bound

def get_planes(n: int, r: int, d: int, use_epsilons=False):
    planes = []
    for r_prime in range(1, r // 2 + 1):
        new_planes = []
        for n_ in generate_admissible_matrices_fixed_r_prime(n, r, r_prime, True):
            if use_epsilons:
                n_ = n_[:, 1:]
            v = r_prime - r * n_.flatten()
            lower, upper = get_plane_intercept_bounds(r_prime - r * n_)
            ks2 = [kp for kp in range(lower + 1, upper) if (kp + r_prime * d) % r == 0]
            if len(ks2) > 0:
                new_planes.append((v, ks2))
        planes += new_planes
    return planes

def get_simplex_inequalities(n: int, r: int):
    A = np.zeros((n * (r + 1), n * r), dtype=int)
    b = np.zeros(n * (r + 1), dtype=int)
    for i in range(n):
        for j in range(r):
            A[i * (r + 1) + j, i * r + j] = -1
            A[i * (r + 1) + j + 1, i * r + j] = 1
    for i in range(n):
        b[i * (r + 1) : (i + 1) * (r + 1)] = 0
        b[i * (r + 1) + r] = 1
    return A, b



## Step 3. Build the Polytope

For given $n$ and $r$, we define the inequalities of the simplex (or product of simplices).  
This forms the initial polytope for the partitioning algorithm.


In [3]:
n = 1
r = 7
A, b = get_simplex_inequalities(n, r - 1)
simplex = Polytope(A, b)
simplex


Polytope(dim=6, n_ineq=7, n_vertices=unknown)

## Step 4. Generate Candidate Hyperplanes
These hyperplanes represent boundaries of stability chambers.


In [4]:
stability_walls = get_planes(n, r, 0, use_epsilons=True)
candidate_hyperplanes = [
    Hyperplane(as_fraction_vector(v), Fraction(k))
    for (v, ks) in stability_walls
    for k in ks
]
len(candidate_hyperplanes)


65


## Step 5. Build the Partition Tree

We now apply the partitioning algorithm to the polytope and the set of candidate hyperplanes.  
The algorithm outputs the total number of chambers and the decision tree representing the partition.


In [5]:
start = perf_counter()
tree, n_chambers = build_partition_tree(simplex, candidate_hyperplanes)
elapsed = perf_counter() - start
print(f"Found {n_chambers} chambers in {elapsed:.2f} s")


Found 1000 chambers...
Found 1296 chambers in 6.46 s


## Step 6. Saving the Tree (Optional)

The decision tree is highly useful in research, for example in identifying **isomorphisms** between stability chambers.  
It can be saved to disk for later analysis.


In [6]:
# save_tree(tree, f"moduli_n{n}_r{r}.json")
